In [22]:
from bs4 import BeautifulSoup
import requests
import urllib 
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
import json
import pandas as pd
import time
import os
import eyed3
import glob
from PIL import Image, ImageFont, ImageDraw
from string import ascii_letters
import textwrap

# User Input

In [23]:
quranLink = "https://mp3quran.net/eng/mrifai" # note: the web page must have 114 ayas
wikipediaLink = "https://en.wikipedia.org/wiki/List_of_chapters_in_the_Quran"
folderName = "Quran Recitations" # folder to save surahs in 

# Scraping Surahs' details from [mp3quran.net](https://mp3quran.net/eng)

## visualizing steps done...

In [24]:
pageHtml = requests.get(quranLink).text
pageHtml = BeautifulSoup(pageHtml, "html.parser")

In [25]:
print(pageHtml.prettify)

<bound method Tag.prettify of <!DOCTYPE html>

<html lang="eng">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<!-- CSRF Token -->
<meta content="E5gCPzhpYjnh7m6EmwfDZjDsIO4JWy6LJ3OF2RbF" name="csrf-token"/>
<title>reciter title hafs</title>
<meta content="reciter title hafs" property="og:title">
<meta content="reciter title hafs" name="twitter:title">
<meta content="reciter title hafs" property="og:title"/>
<meta content="reciter description hafs" name="description"/>
<meta content="reciter description hafs" property="og:description"/>
<meta content="reciter description hafs" name="twitter:description"/>
<meta content="read.keywords-138" name="keywords"/>
<meta content="http://mp3quran.net/eng/mrifai" property="og:url"/>
<meta content="" property="og:site_name"/>
<meta content="" name="twitter:image"/>
<meta content="summary" name="twitter:card">
<meta content="" 

In [26]:
surahsDetails = pageHtml.select("card-sora")
surahsDetails[0:2]

[<card-sora :share=' {"title":"share title read","url":"http:\/\/mp3quran.net\/eng\/mrifai","description":"share description read"} ' :sora='{"id":"001-138","sora_id":1,"sora_num":"001","sora_name":"Al-Fatihah ","read_slug":"mrifai","sora_report":3,"sora_duration":"0:51","sora_link":"http:\/\/mp3quran.net\/eng\/mrifai\/1","sora_audio":"https:\/\/server11.mp3quran.net\/mrifai\/001.mp3","share_url":"http:\/\/mp3quran.net\/eng\/mrifai\/1","share_title":"share title sora","share_description":"share description sora","show":true}' read_id="138" reciter="Mahmood Al rifai" rewaya="Rewayat Hafs A'n Assem"></card-sora>,
 <card-sora :share=' {"title":"share title read","url":"http:\/\/mp3quran.net\/eng\/mrifai","description":"share description read"} ' :sora='{"id":"002-138","sora_id":2,"sora_num":"002","sora_name":"Al-Baqarah ","read_slug":"mrifai","sora_report":2,"sora_duration":"1:53:30","sora_link":"http:\/\/mp3quran.net\/eng\/mrifai\/2","sora_audio":"https:\/\/server11.mp3quran.net\/mrifai\

In [27]:
surahsDetails[0].attrs

{':sora': '{"id":"001-138","sora_id":1,"sora_num":"001","sora_name":"Al-Fatihah ","read_slug":"mrifai","sora_report":3,"sora_duration":"0:51","sora_link":"http:\\/\\/mp3quran.net\\/eng\\/mrifai\\/1","sora_audio":"https:\\/\\/server11.mp3quran.net\\/mrifai\\/001.mp3","share_url":"http:\\/\\/mp3quran.net\\/eng\\/mrifai\\/1","share_title":"share title sora","share_description":"share description sora","show":true}',
 'rewaya': "Rewayat Hafs A'n Assem",
 'read_id': '138',
 'reciter': 'Mahmood Al rifai',
 ':share': ' {"title":"share title read","url":"http:\\/\\/mp3quran.net\\/eng\\/mrifai","description":"share description read"} '}

In [28]:
surahsDetails = pageHtml.select("card-sora")
json.loads(surahsDetails[0].attrs[':sora']) # json.loads() converts string to dictionary

{'id': '001-138',
 'sora_id': 1,
 'sora_num': '001',
 'sora_name': 'Al-Fatihah ',
 'read_slug': 'mrifai',
 'sora_report': 3,
 'sora_duration': '0:51',
 'sora_link': 'http://mp3quran.net/eng/mrifai/1',
 'sora_audio': 'https://server11.mp3quran.net/mrifai/001.mp3',
 'share_url': 'http://mp3quran.net/eng/mrifai/1',
 'share_title': 'share title sora',
 'share_description': 'share description sora',
 'show': True}

# Scraping Surahs' names from [wikipedia](https://en.wikipedia.org/wiki/List_of_chapters_in_the_Quran)

In [29]:
wikiSurahs = pd.read_html(wikipediaLink)[0]


In [30]:
wikiSurahs[wikiSurahs['#'] == 76]

,#,Anglicized title(s),Arabic title(s),English title(s),Number of verses (Number of Rukūʿs),Place of Revelation,Egyptian Standard Chronological Order[5][6][7],Nöldeke's Chronological Order[5],Muqatta'at (isolated letters)[3],Title refers to,Main theme(s),Juz'
75,76,Al-Insaan|Ad-Dahr,ٱلْإِنْسَانal-ʾInsān,"The Human, Man",31 (2),Madinah,98,52,NaN,v. 1 [8],NaN,29


## Adjusting names of some Surahs

In [31]:
wikiSurahs.loc[75, ['Anglicized title(s)']] = 'Al-Insaan' # to remove the "|" because it can't be used in a file name

In [32]:
wikiSurahs[wikiSurahs['#'] == 76] # for debugging by visualizing

,#,Anglicized title(s),Arabic title(s),English title(s),Number of verses (Number of Rukūʿs),Place of Revelation,Egyptian Standard Chronological Order[5][6][7],Nöldeke's Chronological Order[5],Muqatta'at (isolated letters)[3],Title refers to,Main theme(s),Juz'
75,76,Al-Insaan,ٱلْإِنْسَانal-ʾInsān,"The Human, Man",31 (2),Madinah,98,52,NaN,v. 1 [8],NaN,29


In [33]:
wikiSurahs.loc[33, ['Anglicized title(s)']] = 'Saba' # because it had unwanted parentheses

In [34]:
wikiSurahs.loc[93, ['Anglicized title(s)']] = 'Ash-Sharh' # because it had unwanted parentheses

In [35]:
surahDic = json.loads(surahsDetails[0].attrs[':sora'])
mp3FileName = str(surahDic['sora_num']) + " " + wikiSurahs[wikiSurahs['#'] == surahDic['sora_id']]['Anglicized title(s)']
mp3FileName

0    001 Al-Fatihah
Name: Anglicized title(s), dtype: object

In [36]:
type(mp3FileName) # for debugging by visualizing

pandas.core.series.Series

In [37]:
mp3FileName.values[0] + ".mp3" # for debugging by visualizing

'001 Al-Fatihah.mp3'

# Downloading audio files from links found in `surahsDetails` 

In [38]:
session = requests.Session() # These 5 lines are to avoid ConnectionError "attempt failed because the connected party did not properly respond after a period of time"
retry = Retry(connect=3, backoff_factor=0.5) # source: https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [39]:
def save_file_at_dir(dir_path, filename, file_content, mode='wb'): # note: 'wb' not 'w' because we are storing audio not text. source: https://from-locals.com/python-save-file-at-new-dir/
    os.makedirs(dir_path, exist_ok=True)
    with open(os.path.join(dir_path, filename), mode) as f:
        f.write(file_content)

In [40]:
def downloadAudio():
    for i in range(0, len(surahsDetails)):
        surahDic = json.loads(surahsDetails[i].attrs[':sora'])
        mp3FileName = str(surahDic['sora_num']) + " " + wikiSurahs[wikiSurahs['#'] == surahDic['sora_id']]['Anglicized title(s)']
        mp3FileName = mp3FileName.values[0] + ".mp3"
        if (os.path.exists(f'./{folderName}/{mp3FileName}')):
            continue
        mp3Link = surahDic['sora_audio']
        try:
            mp3 = session.get(mp3Link, verify = True) # "verify" argument could be set to False to help fix ConnectionError "attempt failed because the connected party did not properly respond after a period of time"
        except:
            continue
        save_file_at_dir(folderName, mp3FileName, mp3.content) # ".content" accesses the audio of the get request in case you use a link that downloads an mp3 file
        print('.', end = '')
        time.sleep(3) # also to avoid ConnectionError

#downloadAudio()
print()

# Scraping verses in English (with translation) and Arabic from [quran411](https://quran411.com/stacked-view?sn=1) 

## Visualizing the steps...

In [20]:
surahPage = session.get("https://quran411.com/stacked-view?sn=2")
df = pd.read_html(surahPage.content)[0]
df

,0
0,1. Alif-Laaam-MeeemالٓمٓA.L.M.
1,2. Zaalikal Kitaabu laa raiba feeh; hudal lilm...
2,3. Allazeena yu'minoona bilghaibi wa yuqeemoon...
3,4. Wallazeena yu'minoona bimaa unzila ilaika w...
4,5. Ulaaa'ika 'alaa hudam mir rabbihim wa ulaaa...
...,...
281,282. Yaa ayyuhal lazeena aamanoo izaa tadaayan...
282,283. Wa in kuntum 'alaa safarinw wa lam tajido...
283,284. Lillaahi maa fissamaawaati wa maa fil ard...
284,285. Aamanar-Rasoolu bimaaa unzila ilaihi mir-...


In [21]:
df.iloc[6].values[0]

"7. Khatamal laahu 'alaa quloobihim wa 'alaa sam'i-him wa 'alaaa absaarihim ghishaa watunw wa lahum 'azaabun 'azeem (section 1)خَتَمَ اللَّهُ عَلَىٰ قُلُوبِهِمْ وَعَلَىٰ سَمْعِهِمْ ۖ وَعَلَىٰ أَبْصَارِهِمْ غِشَاوَةٌ ۖ وَلَهُمْ عَذَابٌ عَظِيمٌAllah hath set a seal on their hearts and on their hearing, and on their eyes is a veil; great is the penalty they (incur)."

In [41]:
# to do: functionalize code and use it by iterating nested for loops: outer for each audio file (Surah), inner for each row in df
strr = df.iloc[6].values[0]
sentences = ['', '', ''] # 0: first English sentence, 1: Arabic sentence, 2: second English sentence
idx = 0
for char in strr:
    try:
        ascii = ord(char) # returns ASCII, ex: ord('A') == 65
        if (ascii >= 0 and ascii <= ord('z')):
            if (ascii == ord(' ')):
                sentences[idx] += char
            elif (idx == 1):
                idx += 1
                sentences[idx] += char
            else:
                sentences[idx] += char
        else:
            raise
    except:
        idx += (idx == 0)
        sentences[idx] += char
print(sentences)

["7. Khatamal laahu 'alaa quloobihim wa 'alaa sam'i-him wa 'alaaa absaarihim ghishaa watunw wa lahum 'azaabun 'azeem (section 1)", 'خَتَمَ اللَّهُ عَلَىٰ قُلُوبِهِمْ وَعَلَىٰ سَمْعِهِمْ ۖ وَعَلَىٰ أَبْصَارِهِمْ غِشَاوَةٌ ۖ وَلَهُمْ عَذَابٌ عَظِيمٌ', 'Allah hath set a seal on their hearts and on their hearing, and on their eyes is a veil; great is the penalty they (incur).']


In [54]:
eyed3.log.setLevel("ERROR") # to supress warnings related to eyed3

In [42]:
audioFiles = glob.glob(folderName + "/*.mp3")
audioFiles

['Quran Recitations\\001 Al-Fatihah.mp3',
 'Quran Recitations\\002 Al-Baqarah.mp3',
 "Quran Recitations\\003 Aali 'Imran.mp3",
 'Quran Recitations\\004 An-Nisa.mp3',
 "Quran Recitations\\005 Al-Ma'idah.mp3",
 "Quran Recitations\\006 Al-An'am.mp3",
 "Quran Recitations\\007 Al-A'raf.mp3",
 'Quran Recitations\\008 Al-Anfal.mp3',
 'Quran Recitations\\009 At-Tawbah.mp3',
 'Quran Recitations\\010 Yunus.mp3',
 'Quran Recitations\\011 Hud.mp3',
 'Quran Recitations\\012 Yusuf.mp3',
 "Quran Recitations\\013 Ar-Ra'd.mp3",
 'Quran Recitations\\014 Ibrahim.mp3',
 'Quran Recitations\\015 Al-Hijr.mp3',
 'Quran Recitations\\016 An-Nahl.mp3',
 'Quran Recitations\\017 Al-Isra.mp3',
 'Quran Recitations\\018 Al-Kahf.mp3',
 'Quran Recitations\\019 Maryam.mp3',
 'Quran Recitations\\020 Ta-Ha.mp3',
 'Quran Recitations\\021 Al-Anbiya.mp3',
 'Quran Recitations\\022 Al-Hajj.mp3',
 "Quran Recitations\\023 Al-Mu'minun.mp3",
 'Quran Recitations\\024 An-Nur.mp3',
 'Quran Recitations\\025 Al-Furqan.mp3',
 "Quran Rec

## Scraping functions

In [43]:
def extractSentences(fullString):
    sentences = ['', '', ''] # 0: first English sentence, 1: Arabic sentence, 2: second English sentence
    idx = 0
    for char in fullString:
        try:
            ascii = ord(char) # returns ASCII, ex: ord('A') == 65
            if (ascii >= 0 and ascii <= ord('z')):
                if (ascii == ord(' ')):
                    sentences[idx] += char
                elif (idx == 1):
                    idx += 1
                    sentences[idx] += char
                else:
                    sentences[idx] += char
            else:
                raise
        except:
            idx += (idx == 0)
            sentences[idx] += char
    return sentences

In [44]:
def constructTranslation(surahNum):
    surahPage = session.get(f"https://quran411.com/stacked-view?sn={surahNum}")
    df = pd.read_html(surahPage.content)[0]
    fullTranslation = ""
    ayasNum = len(df)
    for index, row in df.iterrows():
        sentences = extractSentences(row[0])
        fullTranslation += f"{sentences[0]}\n{sentences[1]}\n{sentences[2]}"
        if (index != ayasNum-1):
            fullTranslation += "\n\n\n"
    return fullTranslation

In [52]:
def addTranslationToFiles():
    for (i, file) in enumerate(audioFiles):
        f = eyed3.load(file)
        translation = constructTranslation(i+1)
        f.tag.lyrics.set(translation)
        f.tag.save()

In [55]:
addTranslationToFiles()

## Unused: alternative code

In [ ]:
# UNUSED: alternative code for cell above:

strr = df.iloc[0].values[0]
eng1, arabic, eng2 = '', '', ''
travArabic, travEng1 = False, True
for char in strr:
    try:
        ascii = ord(char) # returns ASCII, ex: ord('A') == 65
        if (ascii >= 0 and ascii <= ord('z')):
            if (not travArabic):
                if (travEng1):
                    eng1 += char
                else:
                    eng2 += char
            else:
                if (ascii == ord(' ')):
                    arabic += char
                else:
                    eng2 += char
                    travArabic = False
        else:
            raise
    except:
        arabic += char
        travArabic = True
        travEng1 = False
print(eng1)
print(arabic)
print(eng2)

# Scraping tafsir from [Quran411](https://quran411.com/)

In [184]:
# to do: find a way to get sub pages of the website to obtain strings like "https://quran411.com/surah-fatiha"

# Adding translation, tafsir, and summary images (album art) as metadata to audio files 

In [42]:
audioFiles = glob.glob(folderName + "/*.mp3")
audioFiles

['Quran Recitations\\001 Al-Fatihah.mp3',
 'Quran Recitations\\002 Al-Baqarah.mp3',
 "Quran Recitations\\003 Aali 'Imran.mp3",
 'Quran Recitations\\004 An-Nisa.mp3',
 "Quran Recitations\\005 Al-Ma'idah.mp3",
 "Quran Recitations\\006 Al-An'am.mp3",
 "Quran Recitations\\007 Al-A'raf.mp3",
 'Quran Recitations\\008 Al-Anfal.mp3',
 'Quran Recitations\\009 At-Tawbah.mp3',
 'Quran Recitations\\010 Yunus.mp3',
 'Quran Recitations\\011 Hud.mp3',
 'Quran Recitations\\012 Yusuf.mp3',
 "Quran Recitations\\013 Ar-Ra'd.mp3",
 'Quran Recitations\\014 Ibrahim.mp3',
 'Quran Recitations\\015 Al-Hijr.mp3',
 'Quran Recitations\\016 An-Nahl.mp3',
 'Quran Recitations\\017 Al-Isra.mp3',
 'Quran Recitations\\018 Al-Kahf.mp3',
 'Quran Recitations\\019 Maryam.mp3',
 'Quran Recitations\\020 Ta-Ha.mp3',
 'Quran Recitations\\021 Al-Anbiya.mp3',
 'Quran Recitations\\022 Al-Hajj.mp3',
 "Quran Recitations\\023 Al-Mu'minun.mp3",
 'Quran Recitations\\024 An-Nur.mp3',
 'Quran Recitations\\025 Al-Furqan.mp3',
 "Quran Rec

In [117]:
quranImgs = glob.glob("QuranImages/*") # Source for images: https://www.thelastdialogue.org/quran-summary/
quranImgs

['QuranImages\\001.png',
 'QuranImages\\002.png',
 'QuranImages\\003.png',
 'QuranImages\\004.png',
 'QuranImages\\005.png',
 'QuranImages\\006.png',
 'QuranImages\\007.png',
 'QuranImages\\008.png',
 'QuranImages\\009.png',
 'QuranImages\\010.png',
 'QuranImages\\011.png',
 'QuranImages\\012.png',
 'QuranImages\\013.png',
 'QuranImages\\014.png',
 'QuranImages\\015.png',
 'QuranImages\\016.png',
 'QuranImages\\017.png',
 'QuranImages\\018.png',
 'QuranImages\\019.png',
 'QuranImages\\020.png',
 'QuranImages\\021.png',
 'QuranImages\\022.png',
 'QuranImages\\023.png',
 'QuranImages\\024.png',
 'QuranImages\\025.png',
 'QuranImages\\026.png',
 'QuranImages\\027.png',
 'QuranImages\\028.png',
 'QuranImages\\029.png',
 'QuranImages\\030.png',
 'QuranImages\\031.png',
 'QuranImages\\032.png',
 'QuranImages\\033.png',
 'QuranImages\\034.png',
 'QuranImages\\035.png',
 'QuranImages\\036.png',
 'QuranImages\\037.png',
 'QuranImages\\038.png',
 'QuranImages\\039.png',
 'QuranImages\\040.png',


In [127]:
for file in audioFiles:
    f = eyed3.load(file)
    f.tag.lyrics.set("بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ")
    with open("QuranImages/001.png", "rb") as cover_art:
        f.tag.images.set(3, cover_art.read(), "image/jpeg")
    f.tag.save()
    break